In [17]:
from language_identifier import LanguageIdentifier
from Preprocessing.preprocessing import preprocessing
import numpy as np
import pandas as pd

In [18]:
LangId = LanguageIdentifier()

In [ ]:
all_wrong = pd.DataFrame(columns = ["phrases", "pred", "true"])

languages = ['deu', 'eng', 'fra', 'ita', 'nld', 'spa']
for language in languages:

    with open(f"corpora/{language}_tst.txt", 'r', encoding='utf-8') as file:
        text = preprocessing(file.read())
    phrases = text.split("  ")
    for i, phrase in enumerate(phrases):
        pred = LangId.identify_language(phrase)
        if pred != language: 
            all_wrong.loc[len(all_wrong)] = {"phrases": phrase, "pred" : pred, "true" : language}
            print(i+1)

In [20]:
all_wrong.to_csv("all_wrong_LID.csv", index=False)

In [21]:
all_wrong.shape

(74, 3)